<a href="https://colab.research.google.com/github/nichakornchaisuwan/Project_Boneage/blob/main/Model_Bonet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# Load in relevant libraries, and alias where appropriate
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv) # เป็น library ในการจัดการ dataframe

In [2]:
# Define relevant variables for the ML task
START_EPOCH=0
NUM_EPOCHS=60
LR=0.0001
PATIENCE=2
BATCH_SIZE=20
NUM_WORKERS=0
NUM_GPUS=1
GPUS=0

In [3]:
# Device will determine whether to run the training on GPU or CPU.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
import os # os จัดการเกี่ยวกับไฟล์ต่างๆ ในโฟล์เดอร์

In [5]:
from google.colab import drive # เชื่อม drive ของเรา ถ้าเชื่อมสำเร็จจะขึ้นคำว่าMounted at /content/drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
path = '/content/drive/My Drive/Project_Boneage'  # ชี้ว่า data เราอยู่โฟล์เดอร์ไหน

In [9]:
#loading dataframes # เอาที่อยู่ของไฟล์เชื่อมกับที่อยู่ของไฟล์เรา โดยกำหนดชื่่อไฟล์ที่เราต้องการ
train_path = os.path.join(path,'train.csv')
train_dataset = pd.read_csv(train_path)
validation_path = os.path.join(path,'validation.csv')
validation_dataset = pd.read_csv(validation_path)
test_path = os.path.join(path,'boneage-test-dataset.csv')
test_dataset = pd.read_csv(test_path)

In [19]:

train_dataset

,id,boneage,male
0,1377,180,False
1,1378,12,False
2,1379,94,False
3,1380,120,True
4,1381,82,False
...,...,...,...
10084,13259,94,False
10085,13261,94,False
10086,13274,94,False
10087,13286,94,False


In [20]:

validation_dataset

,id,boneage,male
0,10186,10,True
1,10758,15,False
2,10860,180,False
3,10877,180,False
4,11008,180,False
...,...,...,...
2517,15605,50,False
2518,15606,113,False
2519,15608,55,False
2520,15609,150,True


In [21]:
test_dataset

,Case ID,Sex
0,4360,M
1,4361,M
2,4362,M
3,4363,M
4,4364,M
...,...,...
195,4555,F
196,4556,F
197,4557,F
198,4558,F


In [22]:

#appending file extension to id column for both training and testing dataframes
# ต่อท้าย data ใน column 'id' ด้วย .png
train_dataset['id'] = train_dataset['id'].apply(lambda x: str(x)+'.png') # ทำการเปลี่ยน type ใน column 'id' จาก int  เป็น string เพื่อให้สามารถต่อท้ายด้วย .png ได้ # ฟังก์ชันแลมบ์ดาเป็นฟังก์ชันที่ไม่ระบุตัวตนขนาดเล็ก
validation_dataset['id'] = validation_dataset['id'].apply(lambda x: str(x)+'.png')
test_dataset['Case ID'] = test_dataset['Case ID'].apply(lambda x: str(x)+'.png')

train_dataset.head() # แสดงค่าของ train_df บางส่วน

,id,boneage,male
0,1377.png,180,False
1,1378.png,12,False
2,1379.png,94,False
3,1380.png,120,True
4,1381.png,82,False


In [23]:
#finding out the number of male and female children in the dataset
#creating a new column called gender to keep the gender of the child as a string
train_dataset['gender'] = train_dataset['male'].apply(lambda x: 'male' if x else 'female') # ทำการเปลี่ยนค่าข้อมูล gender จาก true -> male, false -> female 
train_dataset['gender']

0        female
1        female
2        female
3          male
4        female
          ...  
10084    female
10085    female
10086    female
10087    female
10088    female
Name: gender, Length: 10089, dtype: object

In [24]:

validation_dataset['gender'] = validation_dataset['male'].apply(lambda x: 'male' if x else 'female') # ทำการเปลี่ยนค่าข้อมูล gender จาก true -> male, false -> female 
validation_dataset['gender']

0         male
1       female
2       female
3       female
4       female
         ...  
2517    female
2518    female
2519    female
2520      male
2521      male
Name: gender, Length: 2522, dtype: object

In [25]:

#mean age is # หาค่าเฉลี่ยอายุกระดูก
mean_bone_age = train_dataset['boneage'].mean() 
print('mean: ' + str(mean_bone_age))

mean: 127.29467737139458


In [26]:

#standard deviation of boneage # หาค่าส่วนเบี่ยงเบนมาตรฐานของอายุกระดูก
std_bone_age = train_dataset['boneage'].std()
std_bone_age

41.230590949837755

In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from  keras.applications.inception_v3 import preprocess_input 

In [27]:
#models perform better when features are normalised to have zero mean and unity standard deviation
#using z score for the training 
# ทำการหาค่า z-score 
train_dataset[['bone_age_z']] = (train_dataset[['boneage']] - mean_bone_age)/(std_bone_age)
train_dataset

,id,boneage,male,gender,bone_age_z
0,1377.png,180,False,female,1.278306
1,1378.png,12,False,female,-2.796338
2,1379.png,94,False,female,-0.807524
3,1380.png,120,True,male,-0.176924
4,1381.png,82,False,female,-1.098570
...,...,...,...,...,...
10084,13259.png,94,False,female,-0.807524
10085,13261.png,94,False,female,-0.807524
10086,13274.png,94,False,female,-0.807524
10087,13286.png,94,False,female,-0.807524


In [28]:
mean_boneage = validation_dataset['boneage'].mean() 
print('mean: ' + str(mean_boneage))
     

mean: 127.42505947660587


In [29]:

std_boneage = validation_dataset['boneage'].std()
std_boneage

40.99513006091403

In [30]:

validation_dataset[['bone_age_z']] = (validation_dataset[['boneage']] - mean_boneage)/(std_boneage)
validation_dataset
     

,id,boneage,male,gender,bone_age_z
0,10186.png,10,True,male,-2.864366
1,10758.png,15,False,female,-2.742400
2,10860.png,180,False,female,1.282468
3,10877.png,180,False,female,1.282468
4,11008.png,180,False,female,1.282468
...,...,...,...,...,...
2517,15605.png,50,False,female,-1.888640
2518,15606.png,113,False,female,-0.351873
2519,15608.png,55,False,female,-1.766675
2520,15609.png,150,True,male,0.550674


In [31]:

#reducing down the size of the image 
img_size = 500 # ลดขนาดรูปภาพ
     

In [32]:
train_data_generator = ImageDataGenerator(preprocessing_function = preprocess_input)
val_data_generator = ImageDataGenerator(preprocessing_function = preprocess_input)
     

In [33]:
#train data generator
train_generator = train_data_generator.flow_from_dataframe(
    dataframe = train_dataset,
    directory = (os.path.join(path,'boneage_training_dataset/')),
    x_col= 'id',
    y_col= 'bone_age_z',
    class_mode= 'raw',
    start_epoch=0,
    num_epochs=60,
    lr=0.0001,
    patience=2,
    batch_size=20,
    num_workers=0,
    num_gpus=1,
    gpus=0,
    target_size = (img_size, img_size))

Found 10089 validated image filenames.


In [34]:
#validation data generator
val_generator = val_data_generator.flow_from_dataframe(
    dataframe = validation_dataset,
    directory = (os.path.join(path,'boneage_validation_dataset/')),
    x_col = 'id',
    y_col = 'bone_age_z',
    class_mode = 'raw',
    start_epoch=0,
    num_epochs=60,
    lr=0.0001,
    patience=2,
    batch_size=20,
    num_workers=0,
    num_gpus=1,
    gpus=0,
    target_size = (img_size, img_size))

Found 2522 validated image filenames.


In [35]:
#test data generator
test_data_generator = ImageDataGenerator(preprocessing_function = preprocess_input)
test_generator = test_data_generator.flow_from_directory(
    directory = (os.path.join(path,'boneage-test-dataset/')),
    target_size = (img_size,img_size),
    class_mode = 'binary',classes=['.'])
     

Found 200 images belonging to 1 classes.


In [36]:
import re
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.checkpoint as cp
from collections import OrderedDict
import torch.utils.model_zoo as model_zoo
from torch.utils.model_zoo import load_url as load_state_dict_from_url

In [53]:

num_classes = 1

In [38]:
! git clone https://github.com/BCV-Uniandes/Bonet.git
! cd Bonet

Cloning into 'Bonet'...
remote: Enumerating objects: 52, done.
remote: Total 52 (delta 0), reused 0 (delta 0), pack-reused 52
Unpacking objects: 100% (52/52), done.


In [45]:
# Local imports
from Bonet.models.backbone.inception_v3 import InceptionV3

In [46]:
class BoNet(nn.Module):

    def __init__(self, num_class):
        super(BoNet, self).__init__()
        
        # Backbone
        self.inception_v3 = InceptionV3()

        # Gender
        self.gender = DenseLayer(1, 32)
        self.fc_1 = DenseLayer(100384, 1000)
        self.fc_2 = DenseLayer(1000, 1000)
        self.fc_3 = nn.Linear(1000, 1)

    def forward(self, x, y):
        x = self.inception_v3(x)
        y = self.gender(y)
        x = self.fc_1(torch.cat([x, y], 1))
        x = self.fc_2(x)
        x = self.fc_3(x)
        return x

In [47]:
class DenseLayer(nn.Module):

    def __init__(self, in_channels, out_channels):
        super(DenseLayer, self).__init__()
        self.linear = nn.Linear(in_channels, out_channels)

    def forward(self, x):
        x = self.linear(x)
        return F.relu(x, inplace=True)

In [48]:
import Bonet
from Bonet.models.bonet import BoNet

In [54]:
model = BoNet(num_classes)

In [55]:
net = BoNet()

In [56]:
# Set Loss function with criterion
criterion = nn.L1Loss()

In [58]:
import torch.optim as optim

In [64]:
# Set optimizer with optimizer
optimizer = optim.Adam(net.parameters(),lr=0.0001*500)
annealing = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, factor=0.8,patience=2,cooldown=5,
    min_lr=0.00001, eps=0.00001, verbose=True)

In [ ]:
# We use the pre-defined number of epochs to determine how many iterations to train the network on
for epoch in range(num_epochs):
	#Load in the data in batches using the train_loader object
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

            
            


In [ ]:
train_transform = transforms.Compose([transforms.Resize((500, 500)),
                               transforms.RandomAffine(
                                   20, translate=(0.2, 0.2),
                                   scale=(1, 1.2)),
                               transforms.RandomHorizontalFlip(),
                               transforms.ToTensor()])

val_transform = transforms.Compose([transforms.Resize((500, 500)),
                               transforms.ToTensor()])

In [ ]:
#all_transforms = transforms.Compose([transforms.Resize((32,32)),
                                     #transforms.ToTensor(),
                                     #transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                                                          #std=[0.2023, 0.1994, 0.2010])
                                    # ])

In [ ]:
import torchvision

In [ ]:
# Create Training dataset
train_dataset = torchvision.datasets.ImageFolder(path ='/content/drive/My Drive/Project_Boneage/boneage_training_dataset/',transform = train_transform)

TypeError: ignored

In [ ]:
# Create Training dataset
train_dataset = torchvision.datasets.CIFAR10(root = './data',
                                             train = True,
                                             transform = train_transform,
                                             download = True)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data


In [ ]:
# Create Training dataset
train_dataset = torchvision.datasets.ImageNet(mpimg.imread(os.path.join(path,'boneage_training_dataset/'),
                                             transform = train_transform)

# Create Testing dataset
test_dataset = torchvision.datasets.ImageNet(mpimg.imread(os.path.join(path,'boneage-test-dataset/'),
                                            transform = val_transform)

# Instantiate loader objects to facilitate processing
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)


test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)

SyntaxError: ignored

In [ ]:
imagenet_data = torchvision.datasets.ImageNet('path/to/imagenet_root/')
data_loader = torch.utils.data.DataLoader(imagenet_data,
                                          batch_size=4,
                                          shuffle=True,
                                          num_workers=args.nThreads)